In [1]:
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess

from torch import nn
import pandas as pd
import os

from tqdm.auto import tqdm, trange

from sklearn.metrics import classification_report
import json

from torch.utils.data import Dataset
import torch.utils.data as data
from torch import optim
from torch.autograd import Variable
import numpy as np

/homes/yg007/nytimes_project/venv_nyc/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from lavis.models import model_zoo

In [ ]:
print(model_zoo)

# Load the model

In [2]:
# setup device to use
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
model, vis_processors, txt_processors = load_model_and_preprocess(
    name = "blip2_feature_extractor", model_type="pretrain", is_eval=True, device=device
)

# Define the Dataset class

In [37]:
class TwitterCOMMsDataset(Dataset):
    def __init__(self, csv_path, img_dir):
        """
        Args:
            csv_path (string): Path to the {train_completed|val_completed}.csv file.
            image_folder_dir (string): Directory containing the images
        """
        self.df = pd.read_csv(csv_path, index_col=0)
        self.img_dir = img_dir
        
        self.df['exists'] = self.df['filename'].apply(lambda filename: os.path.exists(os.path.join(img_dir, filename)))
        delete_row = self.df[self.df["exists"]==False].index
        self.df = self.df.drop(delete_row)
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        caption = item['full_text']
        img_filename = item['filename']
        topic = item['topic']
        falsified = int(item['falsified'])
        not_falsified = float(not item['falsified'])
#         label = np.array((falsified, not_falsified))
        label = np.array(falsified)
        domain = topic.split('_')[0]
        diff = topic.split('_')[1]
        
        raw_image = Image.open(os.path.join(self.img_dir, img_filename)).convert('RGB')
        image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
        text_input = txt_processors["eval"](caption)
        sample = {"image": image, "text_input": [text_input]}   # image shape: [1, 3, 224, 224]

        features_multimodal = model.extract_features(sample, mode="multimodal")   # [1, 32, 768] ??? image and text might mismatch
        multimodal_emb = features_multimodal.multimodal_embeds[:, 0, :]   # [1, 768]
#         print(multimodal_emb.shape)
#         features_image = model.extract_features(sample, mode="image")
#         features_text = model.extract_features(sample, mode="text")
#         features_image_proj = features_image.image_embeds_proj[:,0,:]   # [1, 256]
#         features_text_proj = features_text.text_embeds_proj[:,0,:]   # [1, 256]
        
#         multimodal_emb = torch.cat((features_image_proj, features_text_proj), 1)
#         multimodal_emb = features_image_proj * features_text_proj   # [1, 256]
#         print(multimodal_emb.shape)

#         similarity = features_image_proj @ features_text_proj.t()

        return {"multimodal_emb": multimodal_emb,
                "topic": topic, 
                "label": label, 
                "domain": domain, 
                "difficulty": diff}
#                "similarity": similarity}
        
        
        
    

In [93]:
# class ClimateAndCovidDataset(Dataset):
#     def __init__(self, csv_path, img_dir):
#         """
#         Args:
#             csv_path (string): Path to the {train_completed|val_completed}.csv file.
#             image_folder_dir (string): Directory containing the images
#         """
#         self.df = pd.read_csv(csv_path, index_col=0)
#         self.img_dir = img_dir
        
#         self.df['exists'] = self.df['filename'].apply(lambda filename: os.path.exists(os.path.join(img_dir, filename)))
#         delete_row = self.df[self.df["exists"]==False].index
#         self.df = self.df.drop(delete_row)
        
#         self.df['is_military'] = self.df['topic'].apply(lambda topic: 'military' in topic)
#         delete_row = self.df[self.df["is_military"]==True].index
#         self.df = self.df.drop(delete_row)
    
#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         item = self.df.iloc[idx]
#         caption = item['full_text']
#         img_filename = item['filename']
#         topic = item['topic']
#         falsified = float(item['falsified'])
#         not_falsified = float(not item['falsified'])
#         label = np.array((falsified, not_falsified))
#         domain = topic.split('_')[0]
#         diff = topic.split('_')[1]
        
#         try:
#             raw_image = Image.open(os.path.join(self.img_dir, img_filename)).convert('RGB')
#             image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
#             text_input = txt_processors["eval"](caption)
#             sample = {"image": image, "text_input": [text_input]}   # image shape: [1, 3, 224, 224]
        
#             features_multimodal = model.extract_features(sample, mode="multimodal")
# #             features_image = model.extract_features(sample, mode="image")
# #             features_text = model.extract_features(sample, mode="text")
# #             print(features_multimodal.multimodal_embeds[:, 0, :].shape)
        
#             return {"multimodal_emb": features_multimodal.multimodal_embeds[:, 0, :],
#                     "topic": topic, 
#                     "label": label, 
#                     "domain": domain, 
#                     "difficulty": diff}
        
#         except IOError as e:
#             print(e)
        
        
        
    

In [39]:
train_data = TwitterCOMMsDataset(csv_path='../data/train_completed.csv',
                                    img_dir='/import/network-temp/yimengg/data/twitter-comms/train/images/train_image_ids')   # took ~one hour to construct the dataset
val_data = TwitterCOMMsDataset(csv_path='../data/val_completed.csv', 
                               img_dir='/import/network-temp/yimengg/data/twitter-comms/images/val_images/val_tweet_image_ids')

In [40]:
train_data.__len__()

2292375

In [46]:
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size=BATCH_SIZE)
val_iterator = data.DataLoader(val_data, 
                               shuffle = False, 
                               batch_size=BATCH_SIZE)

In [47]:
class Net(nn.Module):
    def __init__(self, in_dim, out_dim=2):
        super(Net, self).__init__()
        
        self.fc = nn.Linear(in_dim, out_dim)
        self.in_dim = in_dim
    
    def forward(self, x):
        x = x.view(-1, self.in_dim)
        out = self.fc(x)
        return out
    
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

In [48]:
def normal_init(m, mean, std):
    if isinstance(m, nn.Linear):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

In [49]:
def main():
#     net = Net(256)
    net = Net(768)
    net.cuda()
    net.train()
    net.weight_init(mean=0, std=0.02)
    
    lr = 0.0001
    optimizer = optim.Adam(net.parameters(), lr=lr, betas=(0.5, 0.999), weight_decay=1e-5)
    
    criterion = nn.CrossEntropyLoss()
    criterion.to(device)
 
    EPOCHS = 2
    for epoch in range(EPOCHS):
        total_loss = 0
        num_correct = 0
        total = 0
        for i, batch in tqdm(enumerate(train_iterator, 0), desc='iterations'):
            inputs = batch["multimodal_emb"].to(device)
            labels = batch["label"].to(device)
            inputs, labels = Variable(inputs), Variable(labels)
            
            net.zero_grad()
            y_preds = net(inputs)
            loss = criterion(y_preds, labels)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
            _, top_pred = y_preds.topk(1, 1)
            y = labels.cpu()
            batch_size = y.shape[0]
            top_pred = top_pred.cpu().view(batch_size)
            
#             num_correct += sum(top_pred == y[:, 0]).item()
            num_correct += sum(top_pred == y).item()
            total += batch_size
            
            if i % 50 == 0:
                print("Epoch [%d/%d]: Training accuracy %.2f" % (epoch+1, EPOCHS, num_correct/total))
#                 print(y_preds)
#                 print(top_pred)
#                 print(labels)
#                 print(num_correct)
#                 print(total)

    return net

In [ ]:
net = main()

iterations: 1it [00:07,  7.96s/it]

Epoch [1/2]: Training accuracy 0.38


iterations: 51it [05:40,  6.98s/it]

Epoch [1/2]: Training accuracy 0.57


iterations: 101it [11:34,  9.23s/it]

...

iterations: 801it [1:43:41,  8.24s/it]

Epoch [1/2]: Training accuracy 0.70


iterations: 851it [1:49:59,  7.34s/it]

Epoch [1/2]: Training accuracy 0.70


iterations: 887it [1:54:18,  7.00s/it]

In [ ]:
# for i, batch in tqdm(enumerate(val_iterator, 0), desc='iterations'):
#     num_correct = 0
#     total = 0
#     similarity = batch["similarity"].squeeze()
#     labels = batch["label"]
#     y_preds = torch.zeros_like(labels)
#     y_preds[similarity < 0.5] = 1
    
#     num_correct += sum(y_preds == labels).item()
#     total += BATCH_SIZE
    
#     if i % 50 == 0:
#         print("Accuracy %.2f" % (num_correct/total))
#         print(y_preds)

# Playground

In [ ]:
val_df = pd.read_csv('../data/val_completed.csv', index_col=0)
val_df

In [ ]:
img_dir = '/import/network-temp/yimengg/data/twitter-comms/images/val_images/val_tweet_image_ids'
val_df['exists'] = val_df['filename'].apply(lambda filename: os.path.exists(os.path.join(img_dir, filename)))

In [ ]:
delete_row = val_df[val_df["exists"]==False].index

In [ ]:
val_df = val_df.drop(delete_row)

In [ ]:
val_df